In [1]:
import geopandas as gpd
from pathlib import Path
from processing.parse_data import load_insar_time_series
from scipy.stats import pearsonr
import numpy as np

In [2]:
folder_path = './input_data'
bbox_path = './input_data/bbox.geojson'
reference_epsg = 4326
working_epsg = 3945
columns_to_plot = ['mean_velocity', 'acceleration']

In [3]:
folder = Path(folder_path)
filepaths = [path for path in folder.rglob('*.csv')][:2]
print('Found ' + str(len(filepaths)) + ' files, loading...')
bbox = gpd.read_file(bbox_path)
gdf_infos, df_ts = load_insar_time_series(filepaths, bbox)
print('Loaded ' + str(len(gdf_infos)) + ' insar points of measures.')

Found 2 files, loading...
Loaded 1366 insar points of measures.


In [4]:
df_ts

,pid,start_year,2018-01-06 00:00:00,2018-01-12 00:00:00,2018-01-18 00:00:00,2018-01-24 00:00:00,2018-01-30 00:00:00,2018-02-05 00:00:00,2018-02-11 00:00:00,2018-02-17 00:00:00,...,2023-09-13 00:00:00,2023-09-25 00:00:00,2023-10-07 00:00:00,2023-10-19 00:00:00,2023-10-31 00:00:00,2023-11-12 00:00:00,2023-11-24 00:00:00,2023-12-06 00:00:00,2023-12-18 00:00:00,2023-12-30 00:00:00
477,42Fkx2c3Oe,2018,-0.1,1.1,0.6,0.0,-0.3,0.4,0.1,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
823,42Fkx2Yknd,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.9,-5.5,-5.2,-5.4,-5.4,-6.2,-5.5,-6.1,-6.6,-6.7
894,42Fkx2Z1qf,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-7.3,-8.0,-7.7,-7.6,-7.3,-9.7,-7.5,-7.8,-8.4,-9.5
1111,42Fkx2bVIF,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.1,0.4,0.4,-0.6,-0.0,-1.4,-0.8,0.7,-0.0,-1.1
1112,42Fkx2bVIG,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.7,1.0,1.0,-0.1,-1.1,-1.8,-0.2,-0.3,-0.1,-1.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,42Fkx2Zr00,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-8.1,-12.4,-13.0,-11.9,-13.2,-11.0,-9.4,-10.4,-8.0,-11.2
690,42Fkx2XeaP,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.5,-8.3,1.6,2.0,-4.3,-1.7,-2.3,-2.7,-8.4,-9.5
997,42Fkx2a83P,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-52.1,-49.5,-44.1,-50.5,-49.2,-34.0,-45.9,-46.2,-48.1,-52.8
999,42Fkx2a83X,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.1,11.9,10.5,4.1,6.5,0.5,-1.7,-2.0,-0.2,0.3


In [5]:
# def insar_series_correlations(insar_series, df_comparison):
#     try:
#         df_comparison_series = df_comparison[insar_series.name]
#     except :
#         return np.nan
#
#     min_date = insar_series.index.min()
#     max_date = df_comparison_series.index.max()
#
#     df_comparison_row = df_comparison_series[min_date:max_date]
#     insar_series = insar_series[min_date:max_date]
#
#     insar_series = insar_series.values
#     correlation, _ = pearsonr(df_comparison_row, insar_series)
#
#     return correlation
#
# pearsonr_coeff = df_ts.apply(lambda insar_series: insar_series_correlations(insar_series))